# Image Segmentation using Dense Prediction Transformers

This notebook is based on the github repo on Dense Prediction Transformers: https://github.com/isl-org/DPT


Before you begin, make sure that your Runtime Type is set to GPU. Go to Runtime > Change Runtime Type > GPU

Install necessary libraries

In [1]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.8 MB/s eta 0:00:00


Connect notebook to Google Drive

In [3]:
import os, sys
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
#change this path to your DPT Segmentation folder
os.chdir('/content/drive/MyDrive/Architecture/MIT_SMARCHS/4570_COMPLAB/2023_demo/DPT_Segmentation')

Import necessary libraries

In [7]:
import os
import glob
import cv2
import argparse

import torch
import torch.nn.functional as F

import util.io

from torchvision.transforms import Compose
from dpt.models import DPTSegmentationModel
from dpt.transforms import Resize, NormalizeImage, PrepareForNet

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from os import listdir

List all images in 'input' folder

In [12]:
images = listdir('input')
print(images)

['room.jpg', 'street.jpg', 'building.jpg']


Create empty dataframe of images and visual feature scores

In [22]:
images_df = pd.DataFrame(images)
images_df.columns = ['filename']
images_df.head()

filename
0      room.jpg
1    street.jpg
2  building.jpg

In [23]:
labels = ['wall', 'building', 'sky', 'floor', 'tree', 'ceiling', 'road', 'bed', 'window', 'grass', 'cabinet', 'sidewalk', 'person', 'earth', 'door', 'table', 'mountain', 'plant', 'curtain', 'chair', 'car', 'water', 'painting', 'sofa', 'shelf', 'house', 'sea', 'mirror', 'rug', 'field', 'armchair', 'seat', 'fence', 'desk', 'rock', 'wardrobe', 'lamp', 'bathtub', 'railing', 'cushion', 'base', 'box', 'column', 'signboard', 'dresser', 'counter', 'sand', 'sink', 'skyscraper', 'fireplace', 'refrigerator', 'grandstand', 'path', 'stairs', 'runway', 'case', 'pooltable', 'pillow', 'screen', 'stairway', 'river', 'bridge', 'bookcase', 'blind', 'coffeetable', 'toilet', 'flower', 'book', 'hill', 'bench', 'countertop', 'stove', 'palmtree', 'kitchen', 'computer', 'swivelchair', 'boat', 'bar', 'arcade', 'hut', 'bus', 'towel', 'light', 'truck', 'tower', 'chandelier', 'awning', 'streetlight', 'booth', 'television', 'airplane', 'dirttrack', 'apparel', 'pole', 'land', 'balustrade', 'escalator', 'ottoman', 'bottle', 'sideboard', 'poster', 'stage', 'van', 'ship', 'fountain', 'conveyerbelt', 'canopy', 'washer', 'toy', 'pool', 'stool', 'barrel', 'basket', 'waterfall', 'tent', 'bag', 'motorbike', 'cradle', 'oven', 'ball', 'food', 'step', 'tank', 'brandname', 'microwave', 'pot', 'animal', 'bicycle', 'lake', 'dishwasher', 'screen', 'blanket', 'sculpture', 'hood', 'sconce', 'vase', 'trafficlight', 'tray', 'trashcan', 'fan', 'pier', 'crtscreen', 'plate', 'monitor', 'bulletinboard', 'shower', 'radiator', 'glass', 'clock', 'flag']

In [24]:
for i in range(0,150):
    images_df[labels[i]] = 0

images_df.head()

<ipython-input-24-dc1bf8bb3782>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  images_df[labels[i]] = 0


filename  wall  building  sky  floor  tree  ceiling  road  bed  window  \
0      room.jpg     0         0    0      0     0        0     0    0       0   
1    street.jpg     0         0    0      0     0        0     0    0       0   
2  building.jpg     0         0    0      0     0        0     0    0       0   

   ...  pier  crtscreen  plate  monitor  bulletinboard  shower  radiator  \
0  ...     0          0      0        0              0       0         0   
1  ...     0          0      0        0              0       0         0   
2  ...     0          0      0        0              0       0         0   

   glass  clock  flag  
0      0      0     0  
1      0      0     0  
2      0      0     0  

[3 rows x 150 columns]

Initialize the DPT model for prediction

In [28]:
model_path = "weights/dpt_hybrid-ade20k-53898607.pt"
model_type = 'dpt_hybrid'
optimize = True

In [30]:
#INITIALIZE MODEL - This may take a while. 
"""Run segmentation network

Args:
    input_path (str): path to input folder
    output_path (str): path to output folder
    model_path (str): path to saved model
"""
print("initialize")

# select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: %s" % device)

net_w = net_h = 480

# load network
if model_type == "dpt_large":
    model = DPTSegmentationModel(
        150,
        path=model_path,
        backbone="vitl16_384",
    )
elif model_type == "dpt_hybrid":
    model = DPTSegmentationModel(
        150,
        path=model_path,
        backbone="vitb_rn50_384",
    )
else:
    assert (
        False
    ), f"model_type '{model_type}' not implemented, use: --model_type [dpt_large|dpt_hybrid]"

transform = Compose(
    [
        Resize(
            net_w,
            net_h,
            resize_target=None,
            keep_aspect_ratio=True,
            ensure_multiple_of=32,
            resize_method="minimal",
            image_interpolation_method=cv2.INTER_CUBIC,
        ),
        NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        PrepareForNet(),
    ]
)

model.eval()

if optimize == True and device == torch.device("cuda"):
    model = model.to(memory_format=torch.channels_last)
    model = model.half()

model.to(device)

initialize
device: cuda


DPTSegmentationModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): HybridEmbed(
        (backbone): ResNetV2(
          (stem): Sequential(
            (conv): StdConv2dSame(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
            (norm): GroupNormAct(
              32, 64, eps=1e-05, affine=True
              (drop): Identity()
              (act): ReLU(inplace=True)
            )
            (pool): MaxPool2dSame(kernel_size=(3, 3), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
          )
          (stages): Sequential(
            (0): ResNetStage(
              (blocks): Sequential(
                (0): Bottleneck(
                  (downsample): DownsampleConv(
                    (conv): StdConv2dSame(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                    (norm): GroupNormAct(
                      32, 256, eps=1e-05, affine=True
                      (drop): Identity()
                      (act): 

Loop through the dataframe and populate the feature scores by running the DPT model on each file. 

In [33]:
input_path = "input"
output_path = "output"
savename = "images_scores.csv"

In [35]:
start_index = 0
for i in range(start_index,len(images_df)):
    print("Processing file "+str(i+1)+"/"+str(len(images_df)))
    
    filename = images_df.iloc[i]['filename']
    img_name = input_path+'/'+filename
    print(img_name)

    # input
    img = util.io.read_image(img_name)
    img_input = transform({"image": img})["image"]

    # compute
    with torch.no_grad():
        sample = torch.from_numpy(img_input).to(device).unsqueeze(0)
        if optimize == True and device == torch.device("cuda"):
            sample = sample.to(memory_format=torch.channels_last)
            sample = sample.half()

        out = model.forward(sample)

        prediction = torch.nn.functional.interpolate(
            out, size=img.shape[:2], mode="bicubic", align_corners=False
        )
        prediction = torch.argmax(prediction, dim=1) + 1
        prediction = prediction.squeeze().cpu().numpy()
        
        for j in range(1,151):
            score = prediction[prediction == j].size/prediction.size
            images_df[labels[j-1]].iloc[i] = score
    
    #save file after every 50 images        
    if(i%50) == 0:
        images_df.to_csv(output_path+'/'+savename)
        print("File_Saved")
            
    # save segmentation output image in output folder
    filepath = os.path.join(output_path, os.path.splitext(os.path.basename(img_name))[0])
    util.io.write_segm_img(filepath, img, prediction, alpha=1)

images_df.to_csv(output_path+'/'+savename)
print("File_Saved")
print("finished")  

Processing file 1/3
input/room.jpg
File_Saved
Processing file 2/3
input/street.jpg
Processing file 3/3
input/building.jpg
File_Saved
finished


You're all done! Now check the output folder for the segmented images and the .csv file containing feature scores. 